<a href="https://colab.research.google.com/github/prat-amaze/Word2Vec/blob/main/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import Packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

from tqdm import tqdm

torch.manual_seed(1)

In [ ]:
train_data= pd.read_csv('/content/train.csv')
test_data= pd.read_csv('/content/test.csv')

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2809 entries, 0 to 2808
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2809 non-null   object 
 1   excerpt         2809 non-null   object 
 2   target          2809 non-null   float64
 3   standard_error  2809 non-null   float64
dtypes: float64(2), object(2)
memory usage: 87.9+ KB


In [ ]:
train_data.drop('standard_error', axis=1) #I don't think it's useful

,id,excerpt,target
0,c12129c31,When the young people returned to the ballroom...,-0.340259
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118
3,dd1000b26,And outside before the palace a great garden w...,-1.054013
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197
...,...,...,...
2804,25ca8f498,When you think of dinosaurs and where they liv...,1.711390
2805,2c26db523,So what is a solid? Solids are usually hard be...,0.189476
2806,cd19e2350,The second state of matter we will discuss is ...,0.255209
2807,15e2e9e7a,Solids are shapes that you can actually touch....,-0.215279


In [ ]:
+


Using cpu device


In [ ]:
#torch.set_default_tensor_type("torch.cuda.FloatTensor")

In [ ]:
l= train_data['excerpt']
max_str= max(l).split(" ")
#print(max_str)
pad= len(max_str)
pad

178

In [ ]:
corpus= train_data['excerpt']
corpus

0       When the young people returned to the ballroom...
1       All through dinner time, Mrs. Fayre was somewh...
2       As Roger had predicted, the snow departed as q...
3       And outside before the palace a great garden w...
4       Once upon a time there were Three Bears who li...
                              ...                        
2804    When you think of dinosaurs and where they liv...
2805    So what is a solid? Solids are usually hard be...
2806    The second state of matter we will discuss is ...
2807    Solids are shapes that you can actually touch....
2808    Animals are made of many cells. They eat thing...
Name: excerpt, Length: 2809, dtype: object

In [ ]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)
len(tokenized_corpus)

2809

In [ ]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [ ]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

dx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [ ]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [ ]:
embedding_dims = 5

In [ ]:

W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)


In [ ]:
W2= torch.load('/content/W2.pt')
W2.shape

torch.Size([54125, 5])

In [ ]:

num_epochs = 1
learning_rate = 0.05

for epo in range(num_epochs):
  loss_val = 0
  for data, target in idx_pairs:
      x = Variable(get_input_layer(data)).float()
      y_true = Variable(torch.from_numpy(np.array([target])).long())

      z1 = torch.matmul(W1, x)
      z2 = torch.matmul(W2, z1)

      log_softmax = F.log_softmax(z2, dim=0)

      loss = F.nll_loss(log_softmax.view(1,-1), y_true)
      loss_val += loss.data
      loss.backward()
      W1.data -= learning_rate * W1.grad.data
      W2.data -= learning_rate * W2.grad.data

      W1.grad.data.zero_()
      W2.grad.data.zero_()

  print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 7.789833068847656


In [ ]:
torch.save(W1,"W1.pt")
torch.save(W2,"W2.pt")

In [ ]:
embeddings= {vocabulary[i]:W2[i] for i in range(vocabulary_size)}

In [ ]:
idx_embeddings= {word2idx[vocabulary[i]]:W2[i] for i in range(vocabulary_size)}

In [ ]:
idx_embeddings[0]

tensor([ 0.4366,  0.0293,  0.5612,  1.0510, -0.1444],
       grad_fn=<SelectBackward0>)

In [ ]:
torch.save(idx_embeddings, "idx_embeddings")

In [ ]:
#torch.save(W2,"W2")
torch.save(idx_embeddings, "idx_embeddings.pt")
torch.save(embeddings,"embeddings.pt")

In [ ]:
a= np.ones((1,5))
b=np.ones((1,5))
l=[]